# CCNY Calendar Scraper

This notebook will scrape the CCNY Fall 2021 academic calendar and create a pandas DataFrame with:
- Index: Python date objects
- Column 'dow': Day of the week  
- Column 'text': Calendar event descriptions

We'll use the requests library, BeautifulSoup, and pandas to accomplish this task.
